In [1]:
import tensorflow as tf
import numpy as np
import time
import psutil
import os
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import pandas as pd

In [2]:
# Ensure TensorFlow only uses a limited amount of GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Define batch sizes and number of runs
BATCH_SIZES = [1, 4, 8, 16, 32, 64]
NUM_RUNS = 5  # Runs per batch size

# Load NASNet model
base_model = NASNetMobile(weights='imagenet', input_shape=(224, 224, 3))

# Create a model that outputs intermediate layer results
layer_outputs = [layer.output for layer in base_model.layers]
model = Model(inputs=base_model.input, outputs=layer_outputs)

# Precompile TensorFlow functions for performance
@tf.function
def run_inference(input_tensor):
    return model(input_tensor)

2025-02-05 12:30:35.904888: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-05 12:30:35.904904: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-05 12:30:35.904909: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-02-05 12:30:35.904922: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-05 12:30:35.904931: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Load CIFAR-100 dataset
def load_cifar100(batch_size):
    dataset = tfds.load("cifar100", split="train", as_supervised=True)
    dataset = dataset.map(lambda x, y: (tf.image.resize(x, [224, 224]), y))  # Resize to 224x224
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [4]:
# Function to measure inference time and memory
def benchmark_model(device, batch_size, dataset_iter):
    print(f"\nRunning on {device} with batch size {batch_size}")
    results = []

    for run in range(NUM_RUNS):
        # Load Dataset
        try:
            input_tensor, _ = next(dataset_iter)  # Get real ImageNet images
        except StopIteration:
            dataset_iter = iter(dataset)  # Restart dataset if exhausted
            input_tensor, _ = next(dataset_iter)
        
        with tf.device(device):
            tensor_sizes = {}

            # Warm-up to avoid cold start latency
            _ = run_inference(input_tensor)

            start_time = time.perf_counter()

            # Measure Layer wise tensor size
            output = run_inference(input_tensor)
            
            # Measure per-layer execution time
            for idx, layer in enumerate(model.layers):
                # Calculate tensor size in MB (float32 = 4 bytes)
                tensor_size = np.prod(output[idx].shape) * 4 / (1024 ** 2)
                tensor_sizes[layer.name] = tensor_size
            
            total_time = time.perf_counter() - start_time
    
            results.append({
                "device": device,
                "batch_size": batch_size,
                "total_time": total_time
            })

    return results

## Experiment Logging

In [5]:
# Run benchmarks for CPU and GPU (if available)
devices = ["/CPU:0"]
if gpus:
    devices.append("/GPU:0")

final_results = []

for device in devices:    
    for batch_size in BATCH_SIZES:
        dataset = load_cifar100(batch_size)
        dataset_iter = iter(dataset)
        try:
            tf.profiler.experimental.start(f"logs/Noise/{device[1:4]}/{batch_size}")
            final_results.extend(benchmark_model(device, batch_size, dataset_iter))
        finally:
            tf.profiler.experimental.stop()

2025-02-05 12:30:38.377764: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 12:30:38.377774: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 1


2025-02-05 12:30:39.925688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-05 12:30:40.896447: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 12:30:40.962006: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 12:30:40.962480: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/CPU/1/plugins/profile/2025_02_05_12_30_40/C17586.xplane.pb
2025-02-05 12:30:41.008910: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().


Running on /CPU:0 with batch size 4


2025-02-05 12:30:43.963011: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 12:30:44.043462: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 12:30:44.043618: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/CPU/4/plugins/profile/2025_02_05_12_30_44/C17586.xplane.pb
2025-02-05 12:30:44.092295: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 12:30:44.100219: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 12:

...

2025-02-05 12:32:10.121826: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 12:32:10.139398: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 12:32:10.140417: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/16/plugins/profile/2025_02_05_12_32_10/C17586.xplane.pb
2025-02-05 12:32:10.184816: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 12:32:10.185845: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 12


Running on /GPU:0 with batch size 32


2025-02-05 12:32:34.239167: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 12:32:34.279092: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 12:32:34.280180: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/32/plugins/profile/2025_02_05_12_32_34/C17586.xplane.pb
2025-02-05 12:32:34.359398: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 12:32:34.361635: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 12


Running on /GPU:0 with batch size 64


2025-02-05 12:33:26.284700: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 12:33:26.712308: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 12:33:26.723687: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/64/plugins/profile/2025_02_05_12_33_26/C17586.xplane.pb


## Results

In [6]:
df = pd.DataFrame(final_results)
df.head()

,device,batch_size,total_time
0,/CPU:0,1,0.051766
1,/CPU:0,1,0.050228
2,/CPU:0,1,0.034767
3,/CPU:0,1,0.046638
4,/CPU:0,1,0.031859


In [7]:
df.groupby(['device', 'batch_size']).agg({'total_time': ['min', 'mean', 'var', 'max']})

total_time                              
                         min      mean       var       max
device batch_size                                         
/CPU:0 1            0.031859  0.043052  0.000084  0.051766
       4            0.073920  0.083248  0.000118  0.097852
       8            0.103913  0.128101  0.000362  0.149268
       16           0.192116  0.249096  0.001474  0.296424
       32           0.658166  0.770774  0.008709  0.884986
       64           1.594943  1.723794  0.014607  1.869776
/GPU:0 1            0.083156  0.102858  0.001515  0.172389
       4            0.087358  0.165196  0.011208  0.294188
       8            0.085706  0.411656  0.511807  1.691380
       16           0.090237  0.166147  0.018041  0.402420
       32           0.181739  1.427878  4.699295  5.244562
       64           0.959132  1.302995  0.174002  1.977547